In [1]:
suppressPackageStartupMessages({
    library(readr)
    library(tidyverse)
    library(feather)
    library(mlr)
    library(caret)
    library(parallel)
    library(broom)
    library(multidplyr)
    library(doParallel)
})

read_csv_ <- function(...) suppressMessages(readr::read_csv(...))

ERROR: Error in library(readr): there is no package called 'readr'


In [1]:
.libPaths(c(.libPaths(), 'C:/R/R-3.5.0/library'))

In [2]:
options(repr.matrix.max.rows=1000, repr.matrix.max.cols=200)

In [3]:
source('load-functions.R')

## bureau-data-prep

In [9]:
bbalance <- read_csv_("data/bureau_balance.csv")

bbalance %>% head()

SK_ID_BUREAU,MONTHS_BALANCE,STATUS
5715448,0,C
5715448,-1,C
5715448,-2,C
5715448,-3,C
5715448,-4,C
5715448,-5,C


In [10]:
bbalance_sum <- bbalance %>% 
  arrange(SK_ID_BUREAU, abs(MONTHS_BALANCE)) %>% 
  group_by(SK_ID_BUREAU)  %>% 
  summarize(len_credit = n(),
            avg_of_num_months = mean(abs(MONTHS_BALANCE), na.rm = TRUE),
            num_status = n_distinct(STATUS),
            num_months_closed = sum(STATUS == 'C'),
            most_recent_status = first(STATUS),
            most_frequent_status = Mode(STATUS)) %>% 
  removeConstantFeatures()

In [11]:
write_feather(bbalance_sum, 'data/bbalance_sum.feather')

In [12]:
rm(bbalance); gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2557260,136.6,6853010,366.0,6853010,366.0
Vcells,8684753,66.3,273826238,2089.2,342282776,2611.5


## bureau-data-prep

In [26]:
bureau <- read_csv_("data/bureau.csv")

In [27]:
bureau <- bureau %>% 
  left_join(bbalance_sum, by = "SK_ID_BUREAU")

bureau %>% filter(SK_ID_CURR == 100001) %>% arrange(SK_ID_CURR, abs(DAYS_CREDIT))

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,len_credit,avg_of_num_months,num_status,num_months_closed,most_recent_status,most_frequent_status
100001,5896635,Active,currency 1,-49,0,1778,NA,NA,0,378000,373239.0,0,0,Consumer credit,-16,10822.5,2,0.5,2,0,0,0
100001,5896636,Active,currency 1,-320,0,411,NA,NA,0,168345,110281.5,NA,0,Consumer credit,-10,9364.5,11,5.0,1,0,0,0
100001,5896634,Active,currency 1,-559,0,902,NA,NA,0,337680,113166.0,0,0,Consumer credit,-6,4630.5,19,9.0,3,0,1,0
100001,5896630,Closed,currency 1,-857,0,-492,-553,NA,0,112500,0.0,0,0,Consumer credit,-155,0.0,29,14.0,3,19,C,C
100001,5896632,Closed,currency 1,-879,0,-514,-544,NA,0,91620,0.0,0,0,Consumer credit,-155,0.0,29,14.0,3,18,C,C
100001,5896631,Closed,currency 1,-909,0,-179,-877,NA,0,279720,0.0,0,0,Consumer credit,-155,0.0,30,14.5,2,29,C,C
100001,5896633,Closed,currency 1,-1572,0,-1329,-1328,NA,0,85500,0.0,0,0,Consumer credit,-155,0.0,52,25.5,3,44,C,C


In [30]:
bureau <- bureau %>% 
  arrange(SK_ID_CURR, abs(DAYS_CREDIT)) %>% 
  group_by(SK_ID_CURR) %>% 
  mutate(DAYS_CREDIT_lag_diff = DAYS_CREDIT - dplyr::lead(DAYS_CREDIT)) %>% 
  ungroup()

In [31]:
if("bureau_sum_ratios" %in% ls()) rm(bureau_sum_ratios); gc()

bureau_sum_ratios <- bureau %>% 
  group_by(SK_ID_CURR) %>% 
  summarize(DEBT_TO_CREDIT_RATIO = sum(AMT_CREDIT_SUM_DEBT, na.rm = TRUE)/sum(AMT_CREDIT_SUM, na.rm = TRUE),
            OVERDUE_TO_DEBT_RATIO = sum(AMT_CREDIT_SUM_OVERDUE, na.rm = TRUE)/sum(AMT_CREDIT_SUM_DEBT, na.rm = TRUE))

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2576940,137.7,6853010,366.0,6853010,366.0
Vcells,74215663,566.3,219060990,1671.4,342282776,2611.5


In [32]:
if("bureau_num_sum" %in% ls()) rm(bureau_num_sum); gc()

bureau_num_sum <- bureau %>% 
  arrange(SK_ID_CURR, abs(DAYS_CREDIT)) %>%
  mutate_if(is.character, funs(factor(.) %>% as.integer)) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize_at(.vars = vars(starts_with("DAYS"),
                            starts_with("AMT"),
                            starts_with("CNT"),
                            CREDIT_DAY_OVERDUE,
                            len_credit,
                            avg_of_num_months,
                            starts_with("num_"),
                            starts_with("most_")),
                .funs = agg) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95)

bureau_num_sum %>% head()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2576862,137.7,6853010,366.0,6853010,366.0
Vcells,42105881,321.3,175248792,1337.1,342282776,2611.5


Removing 7 columns: DAYS_CREDIT_lag_diff_last,DAYS_CREDIT_num_nas,DAYS_CREDIT_UPDATE_num_nas,DAYS_CREDIT_lag_diff_num_nas,AMT_CREDIT_SUM_OVERDUE_num_nas,CNT_CREDIT_PROLONG_num_nas,CREDIT_DAY_OVERDUE_num_nas
Warning message in cor(x, use = "complete.obs"):
"the standard deviation is zero"Removing columns with high corr: AMT_CREDIT_SUM_DEBT_sd,AMT_CREDIT_SUM_LIMIT_sd,CNT_CREDIT_PROLONG_sd,DAYS_ENDDATE_FACT_min,AMT_CREDIT_MAX_OVERDUE_max,AMT_CREDIT_SUM_max,AMT_CREDIT_SUM_DEBT_max,AMT_CREDIT_SUM_LIMIT_max,CNT_CREDIT_PROLONG_max,DAYS_ENDDATE_FACT_sum,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_LIMIT_sum,CNT_CREDIT_PROLONG_sum,DAYS_CREDIT_UPDATE_n_distinct,DAYS_CREDIT_lag_diff_n_distinct,avg_of_num_months_n_distinct,DAYS_CREDIT_first,DAYS_CREDIT_ENDDATE_first,DAYS_ENDDATE_FACT_first,DAYS_CREDIT_UPDATE_first,DAYS_ENDDATE_FACT_last,DAYS_ENDDATE_FACT_mean,AMT_CREDIT_MAX_OVERDUE_mean,DAYS_CREDIT_sum,AMT_CREDIT_MAX_OVERDUE_sd,DAYS_CREDIT_n_distinct,DAYS_ENDDATE_FACT_n_distinct,DAYS_CREDIT_ENDDATE_n_di

SK_ID_CURR,DAYS_CREDIT_mean,DAYS_CREDIT_ENDDATE_mean,DAYS_CREDIT_UPDATE_mean,DAYS_CREDIT_lag_diff_mean,AMT_CREDIT_SUM_mean,AMT_CREDIT_SUM_DEBT_mean,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_OVERDUE_mean,AMT_ANNUITY_mean,CNT_CREDIT_PROLONG_mean,CREDIT_DAY_OVERDUE_mean,len_credit_mean,avg_of_num_months_mean,num_status_mean,num_months_closed_mean,most_recent_status_mean,most_frequent_status_mean,DAYS_CREDIT_sd,DAYS_CREDIT_ENDDATE_sd,DAYS_ENDDATE_FACT_sd,DAYS_CREDIT_UPDATE_sd,DAYS_CREDIT_lag_diff_sd,AMT_CREDIT_SUM_sd,AMT_CREDIT_SUM_OVERDUE_sd,AMT_ANNUITY_sd,CREDIT_DAY_OVERDUE_sd,len_credit_sd,avg_of_num_months_sd,num_status_sd,num_months_closed_sd,most_recent_status_sd,most_frequent_status_sd,DAYS_CREDIT_lag_diff_min,AMT_CREDIT_MAX_OVERDUE_min,AMT_CREDIT_SUM_min,AMT_CREDIT_SUM_DEBT_min,AMT_CREDIT_SUM_LIMIT_min,AMT_CREDIT_SUM_OVERDUE_min,AMT_ANNUITY_min,CNT_CREDIT_PROLONG_min,CREDIT_DAY_OVERDUE_min,len_credit_min,num_status_min,num_months_closed_min,most_recent_status_min,most_frequent_status_min,DAYS_CREDIT_ENDDATE_max,DAYS_ENDDATE_FACT_max,DAYS_CREDIT_UPDATE_max,DAYS_CREDIT_lag_diff_max,AMT_CREDIT_SUM_OVERDUE_max,AMT_ANNUITY_max,CREDIT_DAY_OVERDUE_max,len_credit_max,num_status_max,num_months_closed_max,most_recent_status_max,most_frequent_status_max,DAYS_CREDIT_ENDDATE_sum,DAYS_CREDIT_UPDATE_sum,DAYS_CREDIT_lag_diff_sum,AMT_CREDIT_MAX_OVERDUE_sum,AMT_CREDIT_SUM_sum,AMT_CREDIT_SUM_OVERDUE_sum,AMT_ANNUITY_sum,CREDIT_DAY_OVERDUE_sum,len_credit_sum,avg_of_num_months_sum,num_status_sum,num_months_closed_sum,most_recent_status_sum,most_frequent_status_sum,AMT_CREDIT_MAX_OVERDUE_n_distinct,AMT_CREDIT_SUM_n_distinct,AMT_CREDIT_SUM_DEBT_n_distinct,AMT_CREDIT_SUM_LIMIT_n_distinct,AMT_CREDIT_SUM_OVERDUE_n_distinct,AMT_ANNUITY_n_distinct,CNT_CREDIT_PROLONG_n_distinct,CREDIT_DAY_OVERDUE_n_distinct,len_credit_n_distinct,num_status_n_distinct,num_months_closed_n_distinct,most_recent_status_n_distinct,most_frequent_status_n_distinct,DAYS_CREDIT_Mode,DAYS_CREDIT_ENDDATE_Mode,DAYS_ENDDATE_FACT_Mode,DAYS_CREDIT_UPDATE_Mode,AMT_CREDIT_SUM_Mode,AMT_CREDIT_SUM_DEBT_Mode,AMT_CREDIT_SUM_LIMIT_Mode,AMT_CREDIT_SUM_OVERDUE_Mode,AMT_ANNUITY_Mode,CNT_CREDIT_PROLONG_Mode,CREDIT_DAY_OVERDUE_Mode,len_credit_Mode,avg_of_num_months_Mode,num_status_Mode,num_months_closed_Mode,most_recent_status_Mode,most_frequent_status_Mode,DAYS_CREDIT_lag_diff_first,AMT_CREDIT_MAX_OVERDUE_first,AMT_CREDIT_SUM_first,AMT_CREDIT_SUM_DEBT_first,AMT_CREDIT_SUM_LIMIT_first,AMT_CREDIT_SUM_OVERDUE_first,AMT_ANNUITY_first,CNT_CREDIT_PROLONG_first,CREDIT_DAY_OVERDUE_first,len_credit_first,avg_of_num_months_first,num_status_first,num_months_closed_first,most_recent_status_first,most_frequent_status_first,DAYS_CREDIT_last,DAYS_CREDIT_ENDDATE_last,DAYS_CREDIT_UPDATE_last,AMT_CREDIT_MAX_OVERDUE_last,AMT_CREDIT_SUM_last,AMT_CREDIT_SUM_DEBT_last,AMT_CREDIT_SUM_LIMIT_last,AMT_CREDIT_SUM_OVERDUE_last,AMT_ANNUITY_last,CNT_CREDIT_PROLONG_last,CREDIT_DAY_OVERDUE_last,len_credit_last,avg_of_num_months_last,num_status_last,num_months_closed_last,most_recent_status_last,most_frequent_status_last,DAYS_CREDIT_ENDDATE_num_nas,DAYS_ENDDATE_FACT_num_nas,AMT_CREDIT_MAX_OVERDUE_num_nas,AMT_CREDIT_SUM_num_nas,AMT_CREDIT_SUM_DEBT_num_nas,AMT_CREDIT_SUM_LIMIT_num_nas,AMT_ANNUITY_num_nas,len_credit_num_nas,avg_of_num_months_num_nas,num_status_num_nas,num_months_closed_num_nas,most_recent_status_num_nas,most_frequent_status_num_nas
100001,-735.0000,82.42857,-93.14286,253.8333,207623.6,85240.93,0.000,0,3545.357,0,0,24.57143,11.78571,2.428571,15.714286,4.571429,4.428571,489.9425,1032.8593,369.0786,77.20412,233.9790,122544.54451,0,4800.608,0,16.050515,8.025258,0.7867958,16.997199,3.047247,3.207135,22,Inf,85500,0,0,0,0,0,0,2,1,0,1,1,1778,-544,-6,663,0,10822.5,0,52,3,44,7,7,577,-652,1523,0.000,1453365.0,0,24817.5,0,172,82.5,17,110,32,31,0,7,4,1,1,4,1,1,6,3,5,3,2,-49,1778,NA,-155,378000.00,0.0,0,0,0,0,0,29,14.0,3,0,7,7,271,NA,378000.00,373239.0,0.00,0,10822.5,0,0,2,0.5,2,0,1,1,-1572,-1329,-155,NA,85500,0,0,0,0,0,0,52,25.

In [34]:
if("bureau_sum" %in% ls()) rm(bureau_sum); gc()

bureau_sum <- bureau %>% 
  mutate(ACTIVE_LOAN = ifelse(CREDIT_ACTIVE != 'Closed', 1L, 0L)) %>% 
  mutate_if(is.character, funs(factor(.) %>% as.integer)) %>% 
  arrange(SK_ID_CURR, abs(DAYS_CREDIT)) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize(num_previous_applications = n(),
            currency_used = max(CREDIT_CURRENCY),
            most_recent_credit_type = first(CREDIT_TYPE),
            most_frequent_credit_type = Mode(CREDIT_TYPE),
            num_unique_credit_types_used = n_distinct(CREDIT_TYPE, na.rm = TRUE),
            mean_credit_type = mean(CREDIT_TYPE, na.rm = TRUE),
            median_credit_type = median(CREDIT_TYPE, na.rm = TRUE),
            sd_credit_type = sd(CREDIT_TYPE, na.rm = TRUE),
            perc_active_loans = mean(ACTIVE_LOAN, na.rm = TRUE),
            avg_loan_types = n()/n_distinct(CREDIT_TYPE)) %>% 
  left_join(bureau_num_sum,
            by = "SK_ID_CURR") %>% 
  left_join(bureau_sum_ratios,
            by = "SK_ID_CURR")  %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2580777,137.9,6853010,366.0,6853010,366.0
Vcells,79621653,607.5,210397238,1605.3,342282776,2611.5


Warning message in cor(x, use = "complete.obs"):
"the standard deviation is zero"Removing columns with high corr: num_months_closed_max,num_previous_applications,len_credit_min,CNT_CREDIT_PROLONG_n_distinct


In [36]:
write_feather(bureau_sum, "data/bureau_sum.feather")

In [40]:
rm(bbalance_sum, bureau, bureau_num_sum, bureau_sum, bureau_sum_ratios); gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2580577,137.9,6853010,366.0,6853010,366.0
Vcells,6223380,47.5,168317790,1284.2,342282776,2611.5


## cc-balance-data-prep

In [4]:
cc_balance <- read_csv_("data/credit_card_balance.csv")

cc_balance  %>% arrange(SK_ID_CURR, MONTHS_BALANCE ) %>% head(100)

SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
1489396,100006,-6,0.000,270000,NA,0,NA,NA,0.000,NA,0.00,0.00,0.000,0.000,NA,0,NA,NA,0,Active,0,0
1489396,100006,-5,0.000,270000,NA,0,NA,NA,0.000,NA,0.00,0.00,0.000,0.000,NA,0,NA,NA,0,Active,0,0
1489396,100006,-4,0.000,270000,NA,0,NA,NA,0.000,NA,0.00,0.00,0.000,0.000,NA,0,NA,NA,0,Active,0,0
1489396,100006,-3,0.000,270000,NA,0,NA,NA,0.000,NA,0.00,0.00,0.000,0.000,NA,0,NA,NA,0,Active,0,0
1489396,100006,-2,0.000,270000,NA,0,NA,NA,0.000,NA,0.00,0.00,0.000,0.000,NA,0,NA,NA,0,Active,0,0
1489396,100006,-1,0.000,270000,NA,0,NA,NA,0.000,NA,0.00,0.00,0.000,0.000,NA,0,NA,NA,0,Active,0,0
1843384,100011,-75,189000.000,180000,180000,180000,0,0,NA,0.000,0.00,180000.00,189000.000,189000.000,4,4,0,0,NA,Active,0,0
1843384,100011,-74,184568.850,180000,0,0,0,0,9000.000,9000.000,9000.00,180000.00,184568.850,184568.850,0,0,0,0,1,Active,0,0
1843384,100011,-73,181044.540,180000,0,0,0,0,9000.000,9000.000,9000.00,175568.85,181044.540,181044.540,0,0,0,0,2,Active,0,0
1843384,100011,-72,177544.350,180000,0,0,0,0,9000.000,9000.000,9000.00,172044.54,177544.350,177544.350,0,0,0,0,3,Active,0,0


In [5]:
if("cc_balance_num_sum" %in% ls()) rm(cc_balance_num_sum); gc()

cc_balance_num_sum <- cc_balance %>% 
  mutate(credit_util = (AMT_CREDIT_LIMIT_ACTUAL - AMT_BALANCE)/AMT_CREDIT_LIMIT_ACTUAL,
         credit_atm_drawings_util = AMT_DRAWINGS_ATM_CURRENT/AMT_CREDIT_LIMIT_ACTUAL,
         credit_drawings_util = AMT_DRAWINGS_CURRENT/AMT_CREDIT_LIMIT_ACTUAL,
         credit_drawings_other_util = AMT_DRAWINGS_OTHER_CURRENT/AMT_CREDIT_LIMIT_ACTUAL,
         credit_drawings_pos_util = AMT_DRAWINGS_POS_CURRENT/AMT_CREDIT_LIMIT_ACTUAL,
         SK_DPD_FLAG = ifelse(SK_DPD != 0, 1L, 0L),
         SK_DPD_DEF_FLAG = ifelse(SK_DPD_DEF != 0, 1L, 0L)) %>% 
  arrange(SK_ID_CURR, SK_ID_PREV, -MONTHS_BALANCE) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize_at(.vars = vars(MONTHS_BALANCE, 
                            starts_with("AMT_"),
                            starts_with("CNT_"), 
                            starts_with("credit_"),
                            SK_DPD, SK_DPD_DEF,
                            SK_DPD_FLAG, SK_DPD_DEF_FLAG),
               .funs = agg) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(corr = 0.95, except = "SK_ID_CURR")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2555904,136.5,4024965,215.0,4024965,215.0
Vcells,79017489,602.9,172200653,1313.8,171256519,1306.6


Removing 13 columns: MONTHS_BALANCE_num_nas,AMT_BALANCE_num_nas,AMT_CREDIT_LIMIT_ACTUAL_num_nas,AMT_DRAWINGS_CURRENT_num_nas,AMT_PAYMENT_TOTAL_CURRENT_num_nas,AMT_RECEIVABLE_PRINCIPAL_num_nas,AMT_RECIVABLE_num_nas,AMT_TOTAL_RECEIVABLE_num_nas,CNT_DRAWINGS_CURRENT_num_nas,SK_DPD_num_nas,SK_DPD_DEF_num_nas,SK_DPD_FLAG_num_nas,SK_DPD_DEF_FLAG_num_nas
Warning message in cor(x, use = "complete.obs"):
"the standard deviation is zero"Removing columns with high corr: AMT_RECEIVABLE_PRINCIPAL_mean,AMT_RECIVABLE_mean,AMT_TOTAL_RECEIVABLE_mean,AMT_RECEIVABLE_PRINCIPAL_sd,SK_DPD_sd,MONTHS_BALANCE_min,AMT_RECIVABLE_min,AMT_TOTAL_RECEIVABLE_min,AMT_CREDIT_LIMIT_ACTUAL_max,AMT_RECEIVABLE_PRINCIPAL_max,CNT_DRAWINGS_POS_CURRENT_max,CNT_INSTALMENT_MATURE_CUM_max,SK_DPD_max,SK_DPD_DEF_max,MONTHS_BALANCE_sum,AMT_INST_MIN_REGULARITY_sum,AMT_PAYMENT_TOTAL_CURRENT_sum,AMT_RECIVABLE_sum,AMT_TOTAL_RECEIVABLE_sum,CNT_DRAWINGS_POS_CURRENT_sum,CNT_INSTALMENT_MATURE_CUM_sum,SK_DPD_DEF_sum,SK_DPD_FLAG_sum,SK_DPD_DE

In [6]:
cc_balance_feat_sum <- cc_balance %>% 
  arrange(SK_ID_CURR, SK_ID_PREV, -MONTHS_BALANCE) %>% 
  mutate_if(is.character, funs(factor(.) %>% as.integer)) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize(num_prev_apps = n_distinct(SK_ID_PREV),
            num_rows_cc_bal = n(),
            num_contract_status = n_distinct(NAME_CONTRACT_STATUS),
            perc_missed_payments = mean(ifelse(AMT_PAYMENT_CURRENT < AMT_INST_MIN_REGULARITY, 1L, 0L), na.rm = TRUE),
            cash_drawings_ratio = sum(AMT_DRAWINGS_ATM_CURRENT, na.rm = TRUE) / sum(AMT_DRAWINGS_CURRENT, na.rm = TRUE),
            AVG_DRAWINGS_CURRENT = sum(AMT_DRAWINGS_CURRENT, na.rm = TRUE) / sum(CNT_DRAWINGS_CURRENT	, na.rm = TRUE)) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(corr = 0.95, except = "SK_ID_CURR")

Removing columns with high corr: 


In [7]:
cc_balance_sk_prev_sum <- cc_balance %>% 
  arrange(SK_ID_CURR, SK_ID_PREV, -MONTHS_BALANCE) %>% 
  group_by(SK_ID_CURR, SK_ID_PREV) %>% 
  summarize(CNT_INSTALMENT_MATURE_CUM = max(CNT_INSTALMENT_MATURE_CUM, na.rm = TRUE)) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize(INSTALMENTS_PER_LOAN = mean(CNT_INSTALMENT_MATURE_CUM, na.rm = TRUE))

In [9]:
cc_balance_trends_feats <- add_trend_features(cc_balance, 
                                              rec_feat = "MONTHS_BALANCE",
                                              feature_names = c("AMT_BALANCE", "AMT_DRAWINGS_ATM_CURRENT", 
                                                                "CNT_DRAWINGS_ATM_CURRENT",
                                                               "SK_DPD", "SK_DPD_DEF"),
                                             periods = c("all", 6, 12, 24)) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

head(cc_balance_trends_feats)

Trend features for period all have been created for the feature AMT_BALANCE ...
Trend features for period all have been created for the feature AMT_DRAWINGS_ATM_CURRENT ...
Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Trend features for period all have been created for the feature CNT_DRAWINGS_ATM_CURRENT ...
Trend features for period all have been created for the feature SK_DPD ...
Trend features for period all have been created for the feature SK_DPD_DEF ...
Trend features for period 6 have been created for the feature AM

SK_ID_CURR,trend_AMT_BALANCE_all,trend_CNT_DRAWINGS_ATM_CURRENT_all,trend_SK_DPD_all,trend_AMT_DRAWINGS_ATM_CURRENT_6,trend_SK_DPD_6,trend_SK_DPD_DEF_6,trend_AMT_BALANCE_12,trend_AMT_DRAWINGS_ATM_CURRENT_12,trend_CNT_DRAWINGS_ATM_CURRENT_12,trend_SK_DPD_12,trend_SK_DPD_DEF_12,trend_AMT_BALANCE_24,trend_AMT_DRAWINGS_ATM_CURRENT_24,trend_CNT_DRAWINGS_ATM_CURRENT_24,trend_SK_DPD_24
378907,474.8317,2.799552e-05,1.679731e-03,1.900017,0.0016889039,0.0016889039,4936.543,184.4262,0.00273224,0.1639344,0.1639344,2354.761,14.61039,0.0002164502,0.012987013
363914,440.4577,4.119705e-03,3.071417e-04,60.318356,0.0000000000,0.0000000000,2731.486,405.7377,0.03005464,0.0000000,0.0000000,2087.317,477.75974,0.0396103896,0.000000000
371185,4548.1914,2.543530e-02,0.000000e+00,579.976985,0.0000000000,0.0000000000,9104.930,0.0000,0.00000000,0.0000000,0.0000000,8685.670,703.39900,0.0344230353,0.000000000
337855,21532.0562,1.032258e-01,0.000000e+00,371.271511,0.0000000000,0.0000000000,21418.708,448.7705,0.14480874,0.0000000,0.0000000,NA,NA,NA,NA
126868,3879.3643,6.176219e-03,7.010843e-05,155.201668,0.0002999314,0.0002999314,7689.936,245.9016,0.02732240,0.0000000,0.0000000,5875.065,625.32468,0.0266233766,0.004545455
380010,18166.3617,8.365019e-02,5.982906e-02,3660.504202,0.0000000000,0.0000000000,23068.466,4635.6164,0.12328767,0.0000000,0.0000000,NA,NA,NA,NA


In [14]:
names(cc_balance)

[1] "SK_ID_PREV"                 "SK_ID_CURR"                
 [3] "MONTHS_BALANCE"             "AMT_BALANCE"               
 [5] "AMT_CREDIT_LIMIT_ACTUAL"    "AMT_DRAWINGS_ATM_CURRENT"  
 [7] "AMT_DRAWINGS_CURRENT"       "AMT_DRAWINGS_OTHER_CURRENT"
 [9] "AMT_DRAWINGS_POS_CURRENT"   "AMT_INST_MIN_REGULARITY"   
[11] "AMT_PAYMENT_CURRENT"        "AMT_PAYMENT_TOTAL_CURRENT" 
[13] "AMT_RECEIVABLE_PRINCIPAL"   "AMT_RECIVABLE"             
[15] "AMT_TOTAL_RECEIVABLE"       "CNT_DRAWINGS_ATM_CURRENT"  
[17] "CNT_DRAWINGS_CURRENT"       "CNT_DRAWINGS_OTHER_CURRENT"
[19] "CNT_DRAWINGS_POS_CURRENT"   "CNT_INSTALMENT_MATURE_CUM" 
[21] "NAME_CONTRACT_STATUS"       "SK_DPD"                    
[23] "SK_DPD_DEF"

In [10]:
cc_balance_sum <- data.frame(SK_ID_CURR = unique(cc_balance$SK_ID_CURR)) %>% 
  left_join(cc_balance_num_sum, by = "SK_ID_CURR") %>% 
  left_join(cc_balance_feat_sum,
            by = "SK_ID_CURR") %>% 
  left_join(cc_balance_sk_prev_sum,
            by = "SK_ID_CURR") %>% 
  left_join(cc_balance_trends_feats,
            by = "SK_ID_CURR") %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

head(cc_balance_sum)

Warning message in cor(x, use = "complete.obs"):
"the standard deviation is zero"Removing columns with high corr: AMT_DRAWINGS_POS_CURRENT_sum,CNT_DRAWINGS_OTHER_CURRENT_n_distinct,SK_DPD_DEF_n_distinct,credit_drawings_other_util_first,num_rows_cc_bal,AMT_DRAWINGS_CURRENT_mean,AMT_DRAWINGS_CURRENT_min,AMT_DRAWINGS_ATM_CURRENT_min,CNT_DRAWINGS_ATM_CURRENT_min,credit_drawings_util_min,credit_drawings_other_util_sd,CNT_DRAWINGS_CURRENT_Mode,SK_DPD_sum


SK_ID_CURR,AMT_DRAWINGS_ATM_CURRENT_mean,AMT_DRAWINGS_OTHER_CURRENT_mean,AMT_DRAWINGS_POS_CURRENT_mean,AMT_INST_MIN_REGULARITY_mean,AMT_PAYMENT_TOTAL_CURRENT_mean,CNT_DRAWINGS_ATM_CURRENT_mean,CNT_DRAWINGS_OTHER_CURRENT_mean,CNT_DRAWINGS_POS_CURRENT_mean,credit_util_mean,credit_atm_drawings_util_mean,credit_drawings_util_mean,credit_drawings_other_util_mean,credit_drawings_pos_util_mean,SK_DPD_FLAG_mean,SK_DPD_DEF_FLAG_mean,AMT_CREDIT_LIMIT_ACTUAL_sd,AMT_DRAWINGS_ATM_CURRENT_sd,AMT_DRAWINGS_CURRENT_sd,AMT_DRAWINGS_OTHER_CURRENT_sd,AMT_INST_MIN_REGULARITY_sd,AMT_PAYMENT_TOTAL_CURRENT_sd,AMT_TOTAL_RECEIVABLE_sd,CNT_DRAWINGS_ATM_CURRENT_sd,CNT_DRAWINGS_OTHER_CURRENT_sd,CNT_DRAWINGS_POS_CURRENT_sd,credit_util_sd,credit_atm_drawings_util_sd,credit_drawings_util_sd,credit_drawings_pos_util_sd,SK_DPD_DEF_sd,SK_DPD_FLAG_sd,SK_DPD_DEF_FLAG_sd,AMT_CREDIT_LIMIT_ACTUAL_min,AMT_DRAWINGS_OTHER_CURRENT_min,AMT_DRAWINGS_POS_CURRENT_min,AMT_INST_MIN_REGULARITY_min,AMT_PAYMENT_TOTAL_CURRENT_min,AMT_RECEIVABLE_PRINCIPAL_min,CNT_DRAWINGS_OTHER_CURRENT_min,CNT_DRAWINGS_POS_CURRENT_min,CNT_INSTALMENT_MATURE_CUM_min,credit_util_min,credit_atm_drawings_util_min,credit_drawings_other_util_min,credit_drawings_pos_util_min,SK_DPD_DEF_FLAG_min,AMT_DRAWINGS_ATM_CURRENT_max,AMT_DRAWINGS_CURRENT_max,AMT_DRAWINGS_OTHER_CURRENT_max,AMT_DRAWINGS_POS_CURRENT_max,AMT_INST_MIN_REGULARITY_max,AMT_PAYMENT_TOTAL_CURRENT_max,AMT_TOTAL_RECEIVABLE_max,CNT_DRAWINGS_ATM_CURRENT_max,CNT_DRAWINGS_OTHER_CURRENT_max,credit_util_max,credit_atm_drawings_util_max,credit_drawings_util_max,credit_drawings_other_util_max,credit_drawings_pos_util_max,AMT_CREDIT_LIMIT_ACTUAL_sum,AMT_DRAWINGS_ATM_CURRENT_sum,AMT_DRAWINGS_CURRENT_sum,AMT_DRAWINGS_OTHER_CURRENT_sum,AMT_PAYMENT_CURRENT_sum,AMT_RECEIVABLE_PRINCIPAL_sum,CNT_DRAWINGS_ATM_CURRENT_sum,CNT_DRAWINGS_CURRENT_sum,CNT_DRAWINGS_OTHER_CURRENT_sum,credit_util_sum,credit_atm_drawings_util_sum,credit_drawings_util_sum,credit_drawings_other_util_sum,credit_drawings_pos_util_sum,AMT_CREDIT_LIMIT_ACTUAL_n_distinct,AMT_DRAWINGS_CURRENT_n_distinct,AMT_INST_MIN_REGULARITY_n_distinct,AMT_PAYMENT_CURRENT_n_distinct,AMT_PAYMENT_TOTAL_CURRENT_n_distinct,AMT_RECEIVABLE_PRINCIPAL_n_distinct,CNT_DRAWINGS_ATM_CURRENT_n_distinct,CNT_DRAWINGS_CURRENT_n_distinct,CNT_DRAWINGS_POS_CURRENT_n_distinct,credit_atm_drawings_util_n_distinct,credit_drawings_other_util_n_distinct,credit_drawings_pos_util_n_distinct,SK_DPD_FLAG_n_distinct,SK_DPD_DEF_FLAG_n_distinct,AMT_DRAWINGS_ATM_CURRENT_Mode,AMT_DRAWINGS_CURRENT_Mode,AMT_DRAWINGS_POS_CURRENT_Mode,AMT_INST_MIN_REGULARITY_Mode,AMT_PAYMENT_TOTAL_CURRENT_Mode,AMT_RECIVABLE_Mode,CNT_DRAWINGS_ATM_CURRENT_Mode,CNT_DRAWINGS_POS_CURRENT_Mode,CNT_INSTALMENT_MATURE_CUM_Mode,credit_util_Mode,credit_atm_drawings_util_Mode,credit_drawings_util_Mode,credit_drawings_other_util_Mode,credit_drawings_pos_util_Mode,SK_DPD_DEF_Mode,SK_DPD_FLAG_Mode,SK_DPD_DEF_FLAG_Mode,MONTHS_BALANCE_first,AMT_CREDIT_LIMIT_ACTUAL_first,AMT_DRAWINGS_ATM_CURRENT_first,AMT_DRAWINGS_CURRENT_first,AMT_DRAWINGS_OTHER_CURRENT_first,AMT_DRAWINGS_POS_CURRENT_first,AMT_PAYMENT_TOTAL_CURRENT_first,AMT_TOTAL_RECEIVABLE_first,CNT_DRAWINGS_ATM_CURRENT_first,CNT_DRAWINGS_OTHER_CURRENT_first,CNT_DRAWINGS_POS_CURRENT_first,credit_util_first,credit_atm_drawings_util_first,credit_drawings_util_first,credit_drawings_pos_util_first,SK_DPD_first,SK_DPD_DEF_first,SK_DPD_FLAG_first,SK_DPD_DEF_FLAG_first,MONTHS_BALANCE_last,AMT_CREDIT_LIMIT_ACTUAL_last,AMT_DRAWINGS_ATM_CURRENT_last,AMT_DRAWINGS_CURRENT_last,AMT_DRAWINGS_OTHER_CURRENT_last,AMT_DRAWINGS_POS_CURRENT_last,AMT_INST_MIN_REGULARITY_last,AMT_PAYMENT_TOTAL_CURRENT_last,AMT_RECEIVABLE_PRINCIPAL_last,CNT_DRAWINGS_ATM_CURRENT_last,CNT_DRAWINGS_CURRENT_last,CNT_DRAWINGS_OTHER_CURRENT_last,CNT_DRAWINGS_POS_CURRENT_last,credit_util_last,credit_atm_drawings_util_last,credit_drawings_util_last,credit_drawings_other_util_last,credit_drawings_pos_util_last,SK_DPD_last,SK_DPD_DEF_FLAG_last,AMT_PAYMENT_CURRENT_num_nas,CNT_DR

In [11]:
write_feather(cc_balance_sum, "data/cc_balance_sum.feather")

In [12]:
rm(cc_balance, cc_balance_feat_sum, cc_balance_num_sum, cc_balance_sk_prev_sum, cc_balance_sum, cc_balance_trends_feats); gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2590748,138.4,8658306,462.5,30320014,1619.3
Vcells,6241119,47.7,418163570,3190.4,522697844,3987.9


## payments-data-prep

In [13]:
payments <- read_csv_("data/installments_payments.csv")

In [3]:
head(payments %>% arrange(SK_ID_CURR, SK_ID_PREV, -NUM_INSTALMENT_NUMBER), 15)

SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
1369693,100001,2,4,-1619,-1628,17397.900,17397.900
1369693,100001,1,3,-1649,-1660,3951.000,3951.000
1369693,100001,1,2,-1679,-1715,3951.000,3951.000
1369693,100001,1,1,-1709,-1715,3951.000,3951.000
1851984,100001,1,4,-2856,-2856,3980.925,3980.925
1851984,100001,1,3,-2886,-2875,3982.050,3982.050
1851984,100001,1,2,-2916,-2916,3982.050,3982.050
1038818,100002,2,19,-25,-49,53093.745,53093.745
1038818,100002,1,18,-55,-67,9251.775,9251.775
1038818,100002,1,17,-85,-99,9251.775,9251.775


In [14]:
payments_new_feats <- payments %>% 
  mutate(flag_payment_not_in_full = ifelse(AMT_INSTALMENT <= AMT_PAYMENT, 0L, 1L),
         amt_unpaid = AMT_INSTALMENT - AMT_PAYMENT,
         flag_missed_payment = ifelse(DAYS_ENTRY_PAYMENT > DAYS_INSTALMENT, 1L, 0L),
         num_days_late_payment = ifelse(DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT < 0, 0, DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT))

In [15]:
payments_most_rec_loan_group <- payments_new_feats %>% 
  arrange(SK_ID_CURR, -DAYS_INSTALMENT) %>% 
  group_by(SK_ID_CURR) %>% 
  filter(SK_ID_PREV == first(SK_ID_PREV)) %>% 
  ungroup()

In [16]:
payments_most_rec_agg <- payments_most_rec_loan_group %>% 
  group_by(SK_ID_CURR) %>% 
  summarize_at(.vars = vars(starts_with('AMT_'),
                            starts_with('flag'),
                            starts_with('num_', ignore.case = FALSE)),
               .funs = agg_num) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Removing 1 columns: AMT_INSTALMENT_num_nas
Removing columns with high corr: AMT_PAYMENT_mean,AMT_PAYMENT_sd,AMT_PAYMENT_max,AMT_PAYMENT_sum,AMT_PAYMENT_first,amt_unpaid_num_nas,flag_payment_not_in_full_num_nas,flag_missed_payment_num_nas,num_days_late_payment_num_nas


In [17]:
head(payments_most_rec_agg)

SK_ID_CURR,AMT_INSTALMENT_mean,amt_unpaid_mean,flag_payment_not_in_full_mean,flag_missed_payment_mean,num_days_late_payment_mean,AMT_INSTALMENT_sd,amt_unpaid_sd,flag_payment_not_in_full_sd,flag_missed_payment_sd,num_days_late_payment_sd,AMT_INSTALMENT_min,AMT_PAYMENT_min,amt_unpaid_min,flag_payment_not_in_full_min,flag_missed_payment_min,num_days_late_payment_min,AMT_INSTALMENT_max,amt_unpaid_max,flag_payment_not_in_full_max,flag_missed_payment_max,num_days_late_payment_max,AMT_INSTALMENT_sum,amt_unpaid_sum,flag_payment_not_in_full_sum,flag_missed_payment_sum,num_days_late_payment_sum,AMT_INSTALMENT_Mode,AMT_PAYMENT_Mode,amt_unpaid_Mode,flag_payment_not_in_full_Mode,flag_missed_payment_Mode,num_days_late_payment_Mode,AMT_INSTALMENT_first,amt_unpaid_first,flag_payment_not_in_full_first,flag_missed_payment_first,num_days_late_payment_first,AMT_INSTALMENT_last,AMT_PAYMENT_last,amt_unpaid_last,flag_payment_not_in_full_last,flag_missed_payment_last,num_days_late_payment_last,AMT_PAYMENT_num_nas
100001,7312.725,0,0,0.0000000,0.0000000,6723.450,0,0,0.0000000,0.0000000,3951.000,3951.000,0,0,0,0,17397.90,0,0,0,0,29250.90,0,0,0,0,3951.000,3951.000,0,0,0,0,17397.90,0,0,0,0,3951.000,3951.000,0,0,0,0,0
100002,11559.247,0,0,0.0000000,0.0000000,10058.038,0,0,0.0000000,0.0000000,9251.775,9251.775,0,0,0,0,53093.75,0,0,0,0,219625.70,0,0,0,0,9251.775,9251.775,0,0,0,0,53093.75,0,0,0,0,9251.775,9251.775,0,0,0,0,0
100003,164425.333,0,0,0.0000000,0.0000000,174800.392,0,0,0.0000000,0.0000000,98356.995,98356.995,0,0,0,0,560835.36,0,0,0,0,1150977.33,0,0,0,0,98356.995,98356.995,0,0,0,0,560835.36,0,0,0,0,98356.995,98356.995,0,0,0,0,0
100004,7096.155,0,0,0.0000000,0.0000000,3011.872,0,0,0.0000000,0.0000000,5357.250,5357.250,0,0,0,0,10573.97,0,0,0,0,21288.47,0,0,0,0,5357.250,5357.250,0,0,0,0,10573.97,0,0,0,0,5357.250,5357.250,0,0,0,0,0
100005,6240.205,0,0,0.1111111,0.1111111,4281.015,0,0,0.3333333,0.3333333,4813.200,4813.200,0,0,0,0,17656.24,0,0,1,1,56161.85,0,0,1,1,4813.200,4813.200,0,0,0,0,17656.24,0,0,0,0,4813.200,4813.200,0,0,0,0,0
100006,29027.520,0,0,0.0000000,0.0000000,0.000,0,0,0.0000000,0.0000000,29027.520,29027.520,0,0,0,0,29027.52,0,0,0,0,290275.20,0,0,0,0,29027.520,29027.520,0,0,0,0,29027.52,0,0,0,0,29027.520,29027.520,0,0,0,0,0


In [18]:
payments_trend_agg <- add_trend_features(payments_new_feats, rec_feat = "DAYS_INSTALMENT", 
                                         feature_names = "AMT_PAYMENT", periods = c("all", 5, 10, 30, 60)) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Trend features for period all have been created for the feature AMT_PAYMENT ...
Trend features for period 5 have been created for the feature AMT_PAYMENT ...
Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary may be unreliable"Warning message in summary.lm(x):
"essentially perfect fit: summary 

In [19]:
head(payments_trend_agg)

SK_ID_CURR,trend_AMT_PAYMENT_all,trend_AMT_PAYMENT_5,trend_AMT_PAYMENT_10,trend_AMT_PAYMENT_30,trend_AMT_PAYMENT_60
161674,214.8816,256.24444,786.65391,236.53149,238.3295
151639,112.6313,119.42063,367.57426,121.00651,116.6803
193053,3704.7568,NA,NA,NA,NA
199697,722.6389,678.31246,2317.23891,NA,NA
167756,119.8622,92.11443,48.45163,87.77374,NA
164489,1205.4896,1302.66913,650.23886,NA,NA


In [20]:
payments_rec_loans_trend <- payments_new_feats %>% 
  group_by(SK_ID_CURR, SK_ID_PREV) %>% 
  summarize(days_instalment = max(DAYS_INSTALMENT, na.rm = TRUE),
            AMT_PAID = sum(AMT_PAYMENT, na.rm = TRUE)) %>% 
  add_trend_features(rec_feat = "days_instalment", feature_names = "AMT_PAID", periods = c("all", 3, 5, 10)) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Trend features for period all have been created for the feature AMT_PAID ...
Trend features for period 3 have been created for the feature AMT_PAID ...
Trend features for period 5 have been created for the feature AMT_PAID ...
Trend features for period 10 have been created for the feature AMT_PAID ...
Removing columns with high corr: trend_AMT_PAID_5


In [25]:
payments_rec_loans_aggs <- payments_new_feats %>% 
  group_by(SK_ID_CURR, SK_ID_PREV) %>% 
  summarize(days_instalment = max(DAYS_INSTALMENT, na.rm = TRUE),
            AMT_PAID = sum(AMT_PAYMENT, na.rm = TRUE)) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize_at(.vars = vars(days_instalment, AMT_PAID),
               .funs = agg_rec) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Removing columns with high corr: AMT_PAID_max


In [27]:
payments_agg <- payments %>% 
  group_by(SK_ID_CURR) %>% 
  summarize_at(.vars = vars(starts_with("AMT")),
               .funs = agg_rec) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Removing columns with high corr: AMT_PAYMENT_sd,AMT_INSTALMENT_mean,AMT_PAYMENT_max,AMT_PAYMENT_sum


In [26]:
head(payments_rec_loans_aggs)

SK_ID_CURR,days_instalment_mean,AMT_PAID_mean,days_instalment_min,AMT_PAID_min,days_instalment_max,days_instalment_sum,AMT_PAID_sum,days_instalment_sd,AMT_PAID_sd
100001,-2237.5000,20597.96,-2856,11945.03,-1619,-4475,41195.93,874.6911,12237.1
100002,-25.0000,219625.70,-25,219625.70,-25,-25,219625.70,NaN,NaN
100003,-1054.3333,539621.55,-1980,80773.38,-536,-3163,1618864.65,803.5697,551160.6
100004,-724.0000,21288.47,-724,21288.47,-724,-724,21288.47,NaN,NaN
100005,-466.0000,56161.85,-466,56161.85,-466,-466,56161.85,NaN,NaN
100006,-195.6667,335717.80,-425,25091.33,-11,-587,1007153.42,210.5833,335662.8


In [28]:
payments_sum <- data.frame(SK_ID_CURR = unique(payments$SK_ID_CURR)) %>% 
  left_join(payments_most_rec_agg, by = "SK_ID_CURR") %>% 
  left_join(payments_trend_agg, by = "SK_ID_CURR") %>% 
  left_join(payments_rec_loans_aggs, by = "SK_ID_CURR") %>% 
  left_join(payments_agg, by = "SK_ID_CURR") %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Removing columns with high corr: AMT_PAID_sum,AMT_INSTALMENT_sd.y,num_days_late_payment_sum


In [42]:
payments_sum %>% head()

SK_ID_CURR,AMT_INSTALMENT_mean,amt_unpaid_mean,flag_payment_not_in_full_mean,flag_missed_payment_mean,num_days_late_payment_mean,AMT_INSTALMENT_sd.x,amt_unpaid_sd,flag_payment_not_in_full_sd,flag_missed_payment_sd,num_days_late_payment_sd,AMT_INSTALMENT_min.x,AMT_PAYMENT_min.x,amt_unpaid_min,flag_payment_not_in_full_min,flag_missed_payment_min,num_days_late_payment_min,AMT_INSTALMENT_max.x,amt_unpaid_max,flag_payment_not_in_full_max,flag_missed_payment_max,num_days_late_payment_max,AMT_INSTALMENT_sum.x,amt_unpaid_sum,flag_payment_not_in_full_sum,flag_missed_payment_sum,AMT_INSTALMENT_Mode,AMT_PAYMENT_Mode,amt_unpaid_Mode,flag_payment_not_in_full_Mode,flag_missed_payment_Mode,num_days_late_payment_Mode,AMT_INSTALMENT_first,amt_unpaid_first,flag_payment_not_in_full_first,flag_missed_payment_first,num_days_late_payment_first,AMT_INSTALMENT_last,AMT_PAYMENT_last,amt_unpaid_last,flag_payment_not_in_full_last,flag_missed_payment_last,num_days_late_payment_last,AMT_PAYMENT_num_nas,trend_AMT_PAYMENT_60,mean,min,sd,AMT_PAYMENT_mean,AMT_INSTALMENT_min.y,AMT_PAYMENT_min.y,AMT_INSTALMENT_max.y,AMT_INSTALMENT_sum.y
161674,9086.524,0.0000,0.0000000,0.0000000,0.0000000,4842.306,0.0000,0.0000000,0.0000000,0.000000,7374.510,7374.510,0,0,0,0,21070.62,0.00,0,0,0,72692.19,0.00,0,0,7374.510,7374.510,0,0,0,0,21070.620,0.000,0,0,0,7374.510,7374.510,0.00,0,0,0,0,214.8816,141400.15,12931.88,128326.69,12600.014,3647.700,3647.700,117414.00,1272601.40
151639,26197.470,2710.7581,0.2413793,0.1724138,0.3793103,0.000,7820.6991,0.4354942,0.3844259,1.049278,26197.470,130.005,0,0,0,0,26197.47,26067.47,1,1,5,759726.63,78611.99,7,5,26197.470,26197.470,0,0,0,0,26197.470,19.575,1,0,0,26197.470,26197.470,0.00,0,0,0,0,112.6313,291997.85,73848.78,253956.63,9240.438,354.870,5.040,205878.83,1584384.75
193053,11483.070,0.0000,0.0000000,0.3333333,7.6666667,12682.498,0.0000,0.0000000,0.5773503,13.279056,631.035,631.035,0,0,0,0,25425.00,0.00,0,1,23,34449.21,0.00,0,1,8393.175,8393.175,0,0,0,0,8393.175,0.000,0,0,0,25425.000,25425.000,0.00,0,0,0,0,NA,34449.21,34449.21,NaN,11483.070,631.035,631.035,25425.00,34449.21
199697,19277.822,7059.9000,0.6666667,0.5238095,10.1904762,5960.505,10061.7089,0.4830459,0.5117663,18.098671,1350.000,5.400,0,0,0,0,21179.70,21174.30,1,1,74,404834.27,148257.90,14,11,21179.700,21179.700,0,1,1,0,20899.665,0.000,0,0,0,21179.700,21179.700,0.00,0,1,10,0,NA,201287.11,145997.86,78190.81,14910.157,1350.000,5.400,24350.13,550832.13
167756,2513.255,843.6229,0.7058824,0.4117647,1.7058824,507.102,853.5054,0.4696682,0.5072997,2.568188,2390.265,0.585,0,0,0,0,4481.10,2389.68,1,1,8,42725.34,14341.59,12,7,2390.265,2390.265,0,1,0,0,4481.100,0.000,0,0,0,2390.265,0.585,2389.68,1,0,0,0,NA,34920.61,16186.41,22719.24,3492.061,2165.040,0.585,31097.29,123433.51
164489,9429.762,0.0000,0.0000000,0.0000000,0.0000000,13398.148,0.0000,0.0000000,0.0000000,0.000000,5970.375,5970.375,0,0,0,0,57861.18,0.00,0,0,0,141446.43,0.00,0,0,5970.375,5970.375,0,0,0,0,57861.180,0.000,0,0,0,5970.375,5970.375,0.00,0,0,0,0,NA,141446.43,141446.43,NaN,9429.762,5970.375,5970.375,57861.18,141446.43


In [29]:
write_feather(payments_sum, "data/payments_sum.feather")

## pc-balance data-prep

In [30]:
pc_balance <- read_csv_("data/POS_CASH_balance.csv")

head(pc_balance)

SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
1803195,182943,-31,48,45,Active,0,0
1715348,367990,-33,36,35,Active,0,0
1784872,397406,-32,12,9,Active,0,0
1903291,269225,-35,48,42,Active,0,0
2341044,334279,-35,36,35,Active,0,0
2207092,342166,-32,12,12,Active,0,0


In [7]:
pc_balance %>% filter(SK_ID_CURR == "100006") %>% arrange(SK_ID_CURR, -MONTHS_BALANCE)

ERROR: Error in eval(lhs, parent, parent): object 'pc_balance' not found


In [31]:
if("pc_balance_grp" %in% ls()) rm(pc_balance_grp); gc()

pc_balance_grp <- pc_balance %>% 
  arrange(SK_ID_CURR, -MONTHS_BALANCE) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize(pos_cash_remaining_instalments = first(CNT_INSTALMENT_FUTURE),
            num_completed_contracts = sum(NAME_CONTRACT_STATUS == 'Completed'))

head(pc_balance_grp)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2591816,138.5,18974539,1013.4,93298767,4982.7
Vcells,291751116,2225.9,666316864,5083.6,776978090,5927.9


SK_ID_CURR,pos_cash_remaining_instalments,num_completed_contracts
100001,0,2
100002,6,0
100003,0,2
100004,0,1
100005,0,1
100006,3,2


In [32]:
pc_balance_rec_loan_group <- pc_balance %>% 
  arrange(SK_ID_CURR, -MONTHS_BALANCE) %>% 
  group_by(SK_ID_CURR) %>% 
  filter(SK_ID_PREV == first(SK_ID_PREV)) %>% 
  ungroup()

In [33]:
pc_balance_rec_loan_aggv1 <- pc_balance_rec_loan_group %>% 
  mutate(pos_cash_paid_late = as.integer(SK_DPD > 0),
         pos_cash_paid_late_with_tolerance = as.integer(SK_DPD_DEF > 0)) %>% 
  group_by(SK_ID_CURR) %>% 
  summarize_at(.vars = vars(starts_with('pos_cash_')),
               .funs = funs(n(), sum, mean)) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Removing columns with high corr: pos_cash_paid_late_n


In [34]:
pc_balance_rec_loan_aggv2 <- pc_balance_rec_loan_group %>% 
  mutate(pos_cash_paid_late = as.integer(SK_DPD > 0),
         pos_cash_paid_late_with_tolerance = as.integer(SK_DPD_DEF > 0)) %>% 
  group_by(SK_ID_CURR) %>%
  summarize_at(.vars = vars(starts_with('SK_DPD')),
               .funs = agg_rec) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Removing columns with high corr: SK_DPD_DEF_sum,SK_DPD_DEF_mean,SK_DPD_mean,SK_DPD_DEF_sd,SK_DPD_max


In [36]:
pc_balance_trend_feats <- add_trend_features(pc_balance, rec_feat = "MONTHS_BALANCE",
                                             feature_names = c("SK_DPD","SK_DPD_DEF"), 
                                             periods = c("all", 3, 6, 12, 30, 60))

Trend features for period all have been created for the feature SK_DPD ...
Trend features for period all have been created for the feature SK_DPD_DEF ...
Trend features for period 3 have been created for the feature SK_DPD ...
Trend features for period 3 have been created for the feature SK_DPD_DEF ...
Trend features for period 6 have been created for the feature SK_DPD ...
Trend features for period 6 have been created for the feature SK_DPD_DEF ...
Trend features for period 12 have been created for the feature SK_DPD ...
Trend features for period 12 have been created for the feature SK_DPD_DEF ...
Trend features for period 30 have been created for the feature SK_DPD ...
Trend features for period 30 have been created for the feature SK_DPD_DEF ...
Trend features for period 60 have been created for the feature SK_DPD ...
Trend features for period 60 have been created for the feature SK_DPD_DEF ...


In [37]:
pc_balance_sum <- data.frame(SK_ID_CURR = unique(pc_balance$SK_ID_CURR)) %>% 
  left_join(pc_balance_grp, by = "SK_ID_CURR") %>% 
  left_join(pc_balance_rec_loan_aggv1, by = "SK_ID_CURR") %>% 
  left_join(pc_balance_rec_loan_aggv2, by = "SK_ID_CURR")  %>% 
  left_join(pc_balance_trend_feats, by = "SK_ID_CURR") %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Removing columns with high corr: SK_DPD_sum,trend_SK_DPD_60,trend_SK_DPD_DEF_60,trend_SK_DPD_DEF_6,trend_SK_DPD_DEF_3,trend_SK_DPD_DEF_30,trend_SK_DPD_30


In [38]:
pc_balance_sum %>% head()

SK_ID_CURR,pos_cash_remaining_instalments,num_completed_contracts,pos_cash_paid_late_with_tolerance_n,pos_cash_paid_late_sum,pos_cash_paid_late_with_tolerance_sum,pos_cash_paid_late_mean,pos_cash_paid_late_with_tolerance_mean,SK_DPD_min,SK_DPD_DEF_min,SK_DPD_DEF_max,SK_DPD_sd,trend_SK_DPD_all,trend_SK_DPD_DEF_all,trend_SK_DPD_3,trend_SK_DPD_6,trend_SK_DPD_12,trend_SK_DPD_DEF_12
182943,15,1,33,0,0,0,0,0,0,0,0,0.0000000,0.000000000,0.0000000,0.0000000,0.000000000,0
367990,0,2,19,0,0,0,0,0,0,0,0,0.0000000,0.000000000,0.0000000,0.0000000,0.000000000,0
397406,0,6,17,0,0,0,0,0,0,0,0,0.1988529,0.002093068,0.7324125,0.4836616,0.009056005,0
269225,39,5,10,0,0,0,0,0,0,0,0,0.0000000,0.000000000,0.0000000,0.0000000,0.000000000,0
334279,1,4,36,0,0,0,0,0,0,0,0,0.0000000,0.000000000,0.0000000,0.0000000,0.000000000,0
342166,0,3,13,0,0,0,0,0,0,0,0,0.0000000,0.000000000,0.0000000,0.0000000,0.000000000,0


In [39]:
write_feather(pc_balance_sum, 'data/pc_balance_sum.feather')

In [40]:
rm(pc_balance, pc_balance_grp, pc_balance_rec_loan_aggv1, pc_balance_rec_loan_aggv2, pc_balance_rec_loan_group, pc_balance_sum, pc_balance_trend_feats); gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2601746,139.0,13035652,696.2,93298767,4982.7
Vcells,236765782,1806.4,767737826,5857.4,776978090,5927.9


## prev-data-prep

In [4]:
prev <- read_csv_("data/previous_application.csv")

In [22]:
head(prev)

SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
2030495,271877,Consumer loans,1730.43,17145,17145.0,0,17145,SATURDAY,15,Y,1,0,0.1828318,0.8673362,XAP,Approved,-73,Cash through the bank,XAP,NA,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12,middle,POS mobile with interest,365243,-42,300,-42,-37,0
2802425,108129,Cash loans,25188.62,607500,679671.0,NA,607500,THURSDAY,11,Y,1,NA,NA,NA,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36,low_action,Cash X-Sell: low,365243,-134,916,365243,365243,1
2523466,122040,Cash loans,15060.74,112500,136444.5,NA,112500,TUESDAY,11,Y,1,NA,NA,NA,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12,high,Cash X-Sell: high,365243,-271,59,365243,365243,1
2819243,176158,Cash loans,47041.33,450000,470790.0,NA,450000,MONDAY,7,Y,1,NA,NA,NA,XNA,Approved,-512,Cash through the bank,XAP,NA,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12,middle,Cash X-Sell: middle,365243,-482,-152,-182,-177,1
1784265,202054,Cash loans,31924.40,337500,404055.0,NA,337500,THURSDAY,9,Y,1,NA,NA,NA,Repairs,Refused,-781,Cash through the bank,HC,NA,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24,high,Cash Street: high,NA,NA,NA,NA,NA,NA
1383531,199383,Cash loans,23703.93,315000,340573.5,NA,315000,SATURDAY,8,Y,1,NA,NA,NA,Everyday expenses,Approved,-684,Cash through the bank,XAP,Family,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,18,low_normal,Cash X-Sell: low,365243,-654,-144,-144,-137,1


In [6]:
prev_app_agg <- prev %>% 
  group_by(SK_ID_CURR) %>% 
  summarize_at(.vars = vars(starts_with('AMT')),
               .funs = agg_rec) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

head(prev_app_agg)

Removing columns with high corr: AMT_APPLICATION_mean,AMT_GOODS_PRICE_max,AMT_APPLICATION_max,AMT_APPLICATION_sd,AMT_CREDIT_max,AMT_CREDIT_sd,AMT_APPLICATION_sum,AMT_GOODS_PRICE_sum


SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_DOWN_PAYMENT_mean,AMT_GOODS_PRICE_mean,AMT_ANNUITY_min,AMT_APPLICATION_min,AMT_CREDIT_min,AMT_DOWN_PAYMENT_min,AMT_GOODS_PRICE_min,AMT_ANNUITY_max,AMT_DOWN_PAYMENT_max,AMT_ANNUITY_sum,AMT_CREDIT_sum,AMT_DOWN_PAYMENT_sum,AMT_ANNUITY_sd,AMT_DOWN_PAYMENT_sd,AMT_GOODS_PRICE_sd
100001,3951.000,23787.00,2520.00,24835.5,3951.000,24835.5,23787.0,2520.00,24835.50,3951.000,2520,3951.000,23787.0,2520.00,NaN,NaN,NaN
100002,9251.775,179055.00,0.00,179055.0,9251.775,179055.0,179055.0,0.00,179055.00,9251.775,0,9251.775,179055.0,0.00,NaN,NaN,NaN
100003,56553.990,484191.00,3442.50,435436.5,6737.310,68809.5,68053.5,0.00,68809.50,98356.995,6885,169661.970,1452573.0,6885.00,46332.56,4868.43,424161.6
100004,5357.250,20106.00,4860.00,24282.0,5357.250,24282.0,20106.0,4860.00,24282.00,5357.250,4860,5357.250,20106.0,4860.00,NaN,NaN,NaN
100005,4813.200,20076.75,4464.00,44617.5,4813.200,0.0,0.0,4464.00,44617.50,4813.200,4464,4813.200,40153.5,4464.00,NaN,NaN,NaN
100006,23651.175,291695.50,34840.17,408304.9,2482.920,0.0,0.0,2693.34,26912.34,39954.510,66987,141907.050,2625259.5,69680.34,13623.58,45462.48,253670.6


In [7]:
prev_app_feats <- prev %>% 
  arrange(SK_ID_CURR, -DAYS_DECISION) %>% 
  group_by(SK_ID_CURR) %>% 
  filter(SK_ID_PREV == first(SK_ID_PREV)) %>% 
  ungroup() %>%
  mutate(prev_app_approved = as.integer(NAME_CONTRACT_STATUS == "Approved"),
            prev_app_refused = as.integer(NAME_CONTRACT_STATUS == "Refused")) %>% 
  select(SK_ID_CURR, prev_app_approved, prev_app_refused)

In [5]:
prev_n_apps_sum <- rec_n_summary(df = prev, rec_feat = "DAYS_DECISION", 
                                  feature_names = c("CNT_PAYMENT", "NAME_CONTRACT_STATUS"),
                                  num_rec = c(3, 5, 10))

head(prev_n_apps_sum)

Removing columns with high corr: NAME_CONTRACT_STATUS_mean,NAME_CONTRACT_STATUS_max
Removing columns with high corr: NAME_CONTRACT_STATUS_max
Removing columns with high corr: NAME_CONTRACT_STATUS_max


SK_ID_CURR,rec_3_CNT_PAYMENT_mean,rec_3_CNT_PAYMENT_min,rec_3_NAME_CONTRACT_STATUS_min,rec_3_CNT_PAYMENT_max,rec_3_CNT_PAYMENT_sum,rec_3_NAME_CONTRACT_STATUS_sum,rec_3_CNT_PAYMENT_sd,rec_3_NAME_CONTRACT_STATUS_sd,rec_5_CNT_PAYMENT_mean,rec_5_NAME_CONTRACT_STATUS_mean,rec_5_CNT_PAYMENT_min,rec_5_NAME_CONTRACT_STATUS_min,rec_5_CNT_PAYMENT_max,rec_5_CNT_PAYMENT_sum,rec_5_NAME_CONTRACT_STATUS_sum,rec_5_CNT_PAYMENT_sd,rec_5_NAME_CONTRACT_STATUS_sd,rec_10_CNT_PAYMENT_mean,rec_10_NAME_CONTRACT_STATUS_mean,rec_10_CNT_PAYMENT_min,rec_10_NAME_CONTRACT_STATUS_min,rec_10_CNT_PAYMENT_max,rec_10_CNT_PAYMENT_sum,rec_10_NAME_CONTRACT_STATUS_sum,rec_10_CNT_PAYMENT_sd,rec_10_NAME_CONTRACT_STATUS_sd
271877,20.000000,12,1,36,60,5,13.856406,1.154701,20.0,1.666667,12,1,36,60,5,13.856406,1.1547005,20.00000,1.666667,12,1,36,60,5,13.856406,1.1547005
108129,5.333333,0,1,12,16,3,6.110101,0.000000,11.6,1.000000,0,1,36,58,5,14.310835,0.0000000,15.66667,1.000000,0,1,36,94,6,16.219330,0.0000000
122040,8.000000,0,1,12,24,3,6.928203,0.000000,8.0,1.250000,0,1,12,24,5,6.928203,0.5000000,8.00000,1.250000,0,1,12,24,5,6.928203,0.5000000
176158,10.666667,10,1,12,32,7,1.154701,1.154701,11.2,2.600000,10,1,12,56,13,1.095445,0.8944272,13.40000,2.800000,6,1,36,134,28,8.167687,0.6324555
202054,8.333333,6,1,12,25,3,3.214550,0.000000,19.4,1.400000,6,1,36,97,7,15.323185,0.8944272,20.11111,2.200000,6,1,36,181,22,11.072990,1.0327956
199383,12.000000,6,1,18,36,3,6.000000,0.000000,9.6,1.000000,0,1,18,48,5,6.841053,0.0000000,9.60000,1.166667,0,1,18,48,7,6.841053,0.4082483


In [11]:
rm(prev_n_apsps_sum); gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2588556,138.3,5516830,294.7,5516830,294.7
Vcells,68412988,522.0,184498775,1407.7,184481251,1407.5


In [12]:
prev_apps_trend_feats <- add_trend_features(prev, rec_feat = "DAYS_DECISION", 
                                            feature_names = c("AMT_ANNUITY", "AMT_APPLICATION",
                                                              "AMT_GOODS_PRICE", "DAYS_DECISION",
                                                              "CNT_PAYMENT"),
                                            periods = c(15, 30, 60)) %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

Trend features for period 15 have been created for the feature AMT_ANNUITY ...
Trend features for period 15 have been created for the feature AMT_APPLICATION ...
Trend features for period 15 have been created for the feature AMT_GOODS_PRICE ...
Trend features for period 15 have been created for the feature DAYS_DECISION ...
Trend features for period 15 have been created for the feature CNT_PAYMENT ...
Trend features for period 30 have been created for the feature AMT_ANNUITY ...
Trend features for period 30 have been created for the feature AMT_APPLICATION ...
Trend features for period 30 have been created for the feature AMT_GOODS_PRICE ...
Trend features for period 30 have been created for the feature DAYS_DECISION ...
Trend features for period 30 have been created for the feature CNT_PAYMENT ...
Trend features for period 60 have been created for the feature AMT_ANNUITY ...
Trend features for period 60 have been created for the feature AMT_APPLICATION ...
Trend features for period 60

In [13]:
prev_apps_trend_feats %>% filter(!is.na(trend_CNT_PAYMENT_60)) %>% head()

SK_ID_CURR,trend_AMT_ANNUITY_15,trend_AMT_APPLICATION_15,trend_AMT_GOODS_PRICE_15,trend_DAYS_DECISION_15,trend_CNT_PAYMENT_15,trend_AMT_ANNUITY_30,trend_AMT_APPLICATION_30,trend_AMT_GOODS_PRICE_30,trend_DAYS_DECISION_30,trend_CNT_PAYMENT_30,trend_AMT_ANNUITY_60,trend_AMT_APPLICATION_60,trend_AMT_GOODS_PRICE_60,trend_DAYS_DECISION_60,trend_CNT_PAYMENT_60
206862,893.0022,5870.799,7172.208,-154.50645,1.1790323,777.3476,2049.264,9428.609,-27.29318,1.1675439,270.2231,2958.762,6003.086,-6.689067,0.7713911
206783,2072.7672,24842.558,26194.534,-86.34758,0.9919355,1451.4242,16267.348,21526.630,-27.19038,1.6029963,675.5144,3640.635,9271.159,-8.642338,0.7782980
280586,973.2694,6719.569,7981.097,-151.10242,1.1774194,827.0204,1909.282,10181.763,-32.71190,1.2351447,288.7045,3254.131,6257.488,-10.307844,0.7866770
265681,833.8918,10375.185,10375.185,-254.20000,1.4645161,562.8343,7096.243,7096.243,-75.15653,0.7501972,425.6837,5645.348,5924.769,-21.602263,0.4843836
173680,3093.1714,57721.678,84603.878,-47.17823,3.7612293,1757.7681,7693.800,47795.585,-15.48292,2.0906702,909.9203,1302.600,22750.800,-4.685219,0.7529579
156367,2760.1586,42897.364,69171.302,-56.71532,4.3693108,1550.2664,20201.505,41405.860,-24.10661,2.5701279,829.6510,5520.935,22098.711,-8.897101,1.2383948


In [14]:
prev_sum_in_progress_ <- prev_app_agg %>% 
  left_join(prev_app_feats, by = "SK_ID_CURR") %>% 
  left_join(prev_n_apps_sum, by = "SK_ID_CURR") %>% 
  left_join(prev_apps_trend_feats, by = "SK_ID_CURR") %>% 
  removeConstantFeatures() %>% 
  removeCorrelatedVariables(0.95, except = "SK_ID_CURR")

prev_sum_in_progress_  %>% head()

Warning message in cor(x, use = "complete.obs"):
"the standard deviation is zero"Removing columns with high corr: rec_5_NAME_CONTRACT_STATUS_mean,rec_5_CNT_PAYMENT_sd,rec_5_CNT_PAYMENT_max,rec_3_CNT_PAYMENT_sum,rec_3_CNT_PAYMENT_sd,rec_5_CNT_PAYMENT_sum,rec_3_CNT_PAYMENT_mean,AMT_CREDIT_sum,rec_10_NAME_CONTRACT_STATUS_sum


SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_DOWN_PAYMENT_mean,AMT_GOODS_PRICE_mean,AMT_ANNUITY_min,AMT_APPLICATION_min,AMT_CREDIT_min,AMT_DOWN_PAYMENT_min,AMT_GOODS_PRICE_min,AMT_ANNUITY_max,AMT_DOWN_PAYMENT_max,AMT_ANNUITY_sum,AMT_DOWN_PAYMENT_sum,AMT_ANNUITY_sd,AMT_DOWN_PAYMENT_sd,AMT_GOODS_PRICE_sd,prev_app_approved,prev_app_refused,rec_3_CNT_PAYMENT_min,rec_3_NAME_CONTRACT_STATUS_min,rec_3_CNT_PAYMENT_max,rec_3_NAME_CONTRACT_STATUS_sum,rec_3_NAME_CONTRACT_STATUS_sd,rec_5_CNT_PAYMENT_mean,rec_5_CNT_PAYMENT_min,rec_5_NAME_CONTRACT_STATUS_min,rec_5_NAME_CONTRACT_STATUS_sum,rec_5_NAME_CONTRACT_STATUS_sd,rec_10_CNT_PAYMENT_mean,rec_10_NAME_CONTRACT_STATUS_mean,rec_10_CNT_PAYMENT_min,rec_10_NAME_CONTRACT_STATUS_min,rec_10_CNT_PAYMENT_max,rec_10_CNT_PAYMENT_sum,rec_10_CNT_PAYMENT_sd,rec_10_NAME_CONTRACT_STATUS_sd,trend_AMT_ANNUITY_15,trend_AMT_APPLICATION_15,trend_AMT_GOODS_PRICE_15,trend_DAYS_DECISION_15,trend_CNT_PAYMENT_15,trend_AMT_ANNUITY_30,trend_AMT_APPLICATION_30,trend_AMT_GOODS_PRICE_30,trend_DAYS_DECISION_30,trend_CNT_PAYMENT_30,trend_AMT_ANNUITY_60,trend_AMT_APPLICATION_60,trend_AMT_GOODS_PRICE_60,trend_DAYS_DECISION_60,trend_CNT_PAYMENT_60
100001,3951.000,23787.00,2520.00,24835.5,3951.000,24835.5,23787.0,2520.00,24835.50,3951.000,2520,3951.000,2520.00,NaN,NaN,NaN,1,0,8,1,8,1,NaN,8.0,8,1,1,NaN,8,1.000000,8,1,8,8,NaN,NaN,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100002,9251.775,179055.00,0.00,179055.0,9251.775,179055.0,179055.0,0.00,179055.00,9251.775,0,9251.775,0.00,NaN,NaN,NaN,1,0,24,1,24,1,NaN,24.0,24,1,1,NaN,24,1.000000,24,1,24,24,NaN,NaN,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100003,56553.990,484191.00,3442.50,435436.5,6737.310,68809.5,68053.5,0.00,68809.50,98356.995,6885,169661.970,6885.00,46332.56,4868.43,424161.6,1,0,6,1,12,3,0.0000000,10.0,6,1,3,0.0000000,10,1.000000,6,1,12,30,3.464102,0.0000000,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100004,5357.250,20106.00,4860.00,24282.0,5357.250,24282.0,20106.0,4860.00,24282.00,5357.250,4860,5357.250,4860.00,NaN,NaN,NaN,1,0,4,1,4,1,NaN,4.0,4,1,1,NaN,4,1.000000,4,1,4,4,NaN,NaN,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100005,4813.200,20076.75,4464.00,44617.5,4813.200,0.0,0.0,4464.00,44617.50,4813.200,4464,4813.200,4464.00,NaN,NaN,NaN,0,0,12,1,12,3,0.7071068,12.0,12,1,3,0.7071068,12,1.500000,12,1,12,12,NaN,0.7071068,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100006,23651.175,291695.50,34840.17,408304.9,2482.920,0.0,0.0,2693.34,26912.34,39954.510,66987,141907.050,69680.34,13623.58,45462.48,253670.6,1,0,12,1,18,3,0.0000000,22.5,12,1,6,0.4472136,23,1.555556,0,1,48,138,20.228692,0.7264832,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [15]:
write_feather(prev_sum_in_progress_, "prev_sum_in_progress_.feather")

In [17]:
rm(prev_app_agg, prev, prev_app_feats, prev_apps_trend_feats, prev_n_apps_sum); gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2601209,139.0,8511562,454.6,8511562,454.6
Vcells,21990197,167.8,177376822,1353.3,221721027,1691.6


## app-train-and-test data-prep

In [4]:
tr <- read_csv_("data/application_train.csv") 
te <- read_csv_("data/application_test.csv")

In [19]:
tr %>% head()

SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
100002,1,Cash loans,M,N,Y,0,202500,406597.5,24700.5,351000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648,-2120,NA,1,1,0,1,1,0,Laborers,1,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.08303697,0.2629486,0.1393758,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0,0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2,2,2,2,-1134,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
100003,0,Cash loans,F,N,N,0,270000,1293502.5,35698.5,1129500,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186,-291,NA,1,1,0,1,1,0,Core staff,2,1,1,MONDAY,11,0,0,0,0,0,0,School,0.31126731,0.6222458,NA,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0,0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1,0,1,0,-828,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100004,0,Revolving loans,M,Y,Y,0,67500,135000.0,6750.0,135000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260,-2531,26,1,1,1,1,1,0,Laborers,1,2,2,MONDAY,9,0,0,0,0,0,0,Government,NA,0.5559121,0.7295667,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,0,0,0,-815,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [21]:
te %>% head()

SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
100001,Cash loans,F,N,Y,0,135000,568800,20560.5,450000,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170,-812,NA,1,1,0,1,0,1,NA,2,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.7526145,0.7896544,0.1595195,0.0660,0.0590,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0505,NA,NA,0.0672,0.0612,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0526,NA,NA,0.0666,0.0590,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0514,NA,NA,NA,block of flats,0.0392,"Stone, brick",No,0,0,0,0,-1740,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100005,Cash loans,M,N,Y,0,99000,222768,17370.0,180000,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118,-1623,NA,1,1,0,1,0,0,Low-skill Laborers,2,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.5649902,0.2916555,0.4329617,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
100013,Cash loans,M,Y,Y,0,202500,663264,69777.0,630000,NA,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175,-3503,5,1,1,0,1,0,0,Drivers,2,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NA,0.6997868,0.6109913,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,0,0,0,-856,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4
100028,Cash loans,F,N,Y,2,315000,1575000,49018.5,1575000,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.026392,-13976,-1866,-2000,-4208,NA,1,1,0,1,1,0,Sales staff,4,2,2,WEDNESDAY,11,0,0,0,0,0,0,Business Entity Type 3,0.5257340,0.5096771,0.6127042,0.3052,0.1974,0.9970,0.9592,0.1165,0.32,0.2759,0.3750,0.0417,0

In [6]:
app <- tr %>% 
  select(-TARGET) %>% 
  bind_rows(te) %>% 
  arrange(SK_ID_CURR)

head(app)

SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
100001,Cash loans,F,N,Y,0,135000,568800.0,20560.5,450000,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170,-812,NA,1,1,0,1,0,1,NA,2,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.75261449,0.7896544,0.1595195,0.0660,0.0590,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0505,NA,NA,0.0672,0.0612,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0526,NA,NA,0.0666,0.0590,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0514,NA,NA,NA,block of flats,0.0392,"Stone, brick",No,0,0,0,0,-1740,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100002,Cash loans,M,N,Y,0,202500,406597.5,24700.5,351000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648,-2120,NA,1,1,0,1,1,0,Laborers,1,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.08303697,0.2629486,0.1393758,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0,0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2,2,2,2,-1134,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
100003,Cash loans,F,N,N,0,270000,1293502.5,35698.5,1129500,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186,-291,NA,1,1,0,1,1,0,Core staff,2,1,1,MONDAY,11,0,0,0,0,0,0,School,0.31126731,0.6222458,NA,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0,0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,N

In [7]:
app <- app %>% 
  mutate(annuity_income_ratio = AMT_ANNUITY / AMT_INCOME_TOTAL,
         car_age_ratio = OWN_CAR_AGE / DAYS_BIRTH,
         days_employed_birth_ratio = DAYS_EMPLOYED / DAYS_BIRTH,
         car_employment_ratio = OWN_CAR_AGE / DAYS_EMPLOYED,
         perc_children = CNT_CHILDREN / CNT_FAM_MEMBERS,
         credit_annuity_ratio = AMT_CREDIT / AMT_ANNUITY,
         credit_goods_ratio = AMT_CREDIT / AMT_GOODS_PRICE,
         credit_income_ratio = AMT_CREDIT / AMT_INCOME_TOTAL,
         income_per_child = AMT_INCOME_TOTAL / (1 + CNT_CHILDREN),
         income_per_person = AMT_INCOME_TOTAL / CNT_FAM_MEMBERS,
         phone_change_birth_ratio = DAYS_LAST_PHONE_CHANGE / DAYS_BIRTH,
         phone_change_employment_ratio = DAYS_LAST_PHONE_CHANGE / DAYS_EMPLOYED)

In [12]:
app <- app %>% 
  mutate(ext_sources_min = apply(.[,c("EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3")], 1, min, na.rm = TRUE),
         ext_sources_max = apply(.[,c("EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3")], 1, max, na.rm = TRUE),
         ext_sources_sum = apply(.[,c("EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3")], 1, sum, na.rm = TRUE),
         ext_sources_mean = apply(.[,c("EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3")], 1, mean, na.rm = TRUE),
         ext_sources_median = apply(.[,c("EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3")], 1, median, na.rm = TRUE))

head(app)

Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i]

"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min; returning Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to min

"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing arguments to max; returning -Inf"Warning message in FUN(newX[, i], ...):
"no non-missing argum

SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,annuity_income_ratio,car_age_ratio,days_employed_birth_ratio,car_employment_ratio,perc_children,credit_annuity_ratio,credit_goods_ratio,credit_income_ratio,income_per_child,income_per_person,phone_change_birth_ratio,phone_change_employment_ratio,ext_sources_min,ext_sources_max,ext_sources_sum,ext_sources_mean,ext_sources_median
100001,Cash loans,F,N,Y,0,135000,568800.0,20560.5,450000,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170,-812,NA,1,1,0,1,0,1,NA,2,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.75261449,0.7896544,0.1595195,0.0660,0.0590,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0505,NA,NA,0.0672,0.0612,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0526,NA,NA,0.0666,0.0590,0.9732,NA,NA,NA,0.1379,0.1250,NA,NA,NA,0.0514,NA,NA,NA,block of flats,0.0392,"Stone, brick",No,0,0,0,0,-1740,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1523000,NA,0.12104360,NA,0,27.66470,1.264000,4.213333,135000,67500,0.09043189,0.7471018,0.15951954,0.7896544,1.7017884,0.5672628,0.7526145
100002,Cash loans,M,N,Y,0,202500,406597.5,24700.5,351000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648,-2120,NA,1,1,0,1,1,0,Laborers,1,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.08303697,0.2629486,0.1393758,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0,0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2,2,2,2,-1134,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.1219778,NA,0.06732903,NA,0,16.46110,1.158397,2.007889,202500,202500,0.11986048,1.7802198,0.

In [13]:
write_feather(app, 'data/app.feather')

In [12]:
bbalance_sum <- read_feather('data/bbalance_sum.feather')
bureau_sum <- read_feather("data/bureau_sum.feather")
cc_balance_sum <- read_feather('data/cc_balance_sum.feather')
payments_sum <- read_feather('data/payments_sum.feather')
pc_balance_sum <- read_feather('data/pc_balance_sum.feather')
prev_sum <- read_feather('prev_sum_in_progress_.feather')
app <- read_feather('data/app.feather')

In [13]:
final_df <- data.frame(SK_ID_CURR = unique(app$SK_ID_CURR)) %>% 
  left_join(bureau_sum, by = 'SK_ID_CURR') %>%
  left_join(cc_balance_sum, by = 'SK_ID_CURR') %>%
  left_join(payments_sum, by = 'SK_ID_CURR') %>%
  left_join(pc_balance_sum, by = 'SK_ID_CURR') %>%
  left_join(prev_sum, by = 'SK_ID_CURR') %>%
  left_join(app, by = 'SK_ID_CURR')  %>% 
  removeConstantFeatures()

In [9]:
write_feather(final_df, 'data/final_df.feather')

In [12]:
dim(final_df)

[1] 356255    575

In [5]:
final_df <- read_feather('data/final_df.feather')

In [6]:
n_distinct(final_df$SK_ID_CURR)

[1] 356255

In [14]:
trn <- final_df %>% 
  inner_join(tr %>% select(SK_ID_CURR, TARGET), by = "SK_ID_CURR") %>% 
  arrange(SK_ID_CURR)

In [9]:
n_distinct(tr$SK_ID_CURR)

[1] 307511

In [15]:
tst <- final_df %>% 
  filter(SK_ID_CURR %in% te$SK_ID_CURR) %>% 
  arrange(SK_ID_CURR)

In [16]:
write_feather(trn, 'data/trn.feather')
write_feather(tst, 'data/tst.feather')